In [ ]:
import torch
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Funzione per caricare il modello
def load_model(model_path):
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    # Carica il modello da file .pth
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

In [ ]:

# Funzione per eseguire il modello su un'immagine
def get_predictions(model, image, threshold=0.5):
    # Trasformazione dell'immagine in formato tensor
    transform = transforms.ToTensor()
    image_tensor = transform(image).unsqueeze(0)  # Aggiungi dimensione batch
    with torch.no_grad():
        predictions = model(image_tensor)
    
    # Estrai le bounding box, punteggi e etichette
    boxes = predictions[0]['boxes']
    scores = predictions[0]['scores']
    labels = predictions[0]['labels']

    # Filtra per score > threshold
    keep = scores > threshold
    boxes = boxes[keep]
    labels = labels[keep]
    return boxes, labels

In [ ]:
# Funzione per disegnare le bounding box su un'immagine
def draw_boxes(image, boxes, labels):
    image = np.array(image)
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box.tolist()
        # Disegna la bounding box (etichetta è ignorata in questo esempio)
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    return image

In [ ]:
# Funzione principale per eseguire tutto il processo
def process_images(model_path, images_folder, output_folder):
    model = load_model(model_path)

    # Crea la cartella di output se non esiste
    os.makedirs(output_folder, exist_ok=True)

    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        if image_path.endswith(('.jpg', '.png', '.jpeg')):
            print(f"Processing {image_name}")
            # Carica l'immagine
            image = Image.open(image_path).convert("RGB")
            boxes, labels = get_predictions(model, image)

            # Disegna le bounding box sull'immagine
            result_image = draw_boxes(image, boxes, labels)

            # Salva l'immagine con le bounding box
            result_image_path = os.path.join(output_folder, f"output_{image_name}")
            cv2.imwrite(result_image_path, result_image)
            print(f"Saved output to {result_image_path}")

In [ ]:
def main():
    model_path = '/content/drive/MyDrive/pth_cigarette_detect/best_model.pth'  # Sostituisci con il percorso del tuo modello .pth
    images_folder = '/content/drive/MyDrive/Photo/test'  # Sostituisci con la cartella contenente le immagini
    output_folder = '/content/drive/MyDrive/test_trained'  # Sostituisci con la cartella dove salvare le immagini con le bounding box

    # Esegui il processo di rilevamento
    process_images(model_path, images_folder, output_folder)

if __name__ == "__main__":
    main()